In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('seattle-weather.csv')
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values


category_mapping = {category: idx for idx, category in enumerate(set(y), start=0)}

y = [category_mapping[weather] for weather in y]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 0)

sc = StandardScaler()
train_data = sc.fit_transform(X_train)
test_data = sc.transform(X_test)

train_data, train_labels = torch.tensor(train_data).float(), torch.tensor(y_train)
test_data, test_labels = torch.tensor(test_data).float(), torch.tensor(y_test)

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 5)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = MLP()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-3)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=10)

patience = 20
best_loss = float('inf')
counter = 0

selected_epochs = []
selected_losses = []

prev_lr = optimizer.param_groups[0]['lr']

num_epochs = 5000
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    outputs = model(train_data)
    loss = criterion(outputs, train_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    epoch_loss = loss.item()

    current_lr = optimizer.param_groups[0]['lr']
    if current_lr != prev_lr:
        print(f"Το learning rate άλλαξε από {prev_lr:.6f} σε {current_lr:.6f} στο τέλος της εποχής {epoch}")
        prev_lr = current_lr

    if (epoch + 1) <= 20 or (epoch + 1) % 50 == 0:
        selected_epochs.append(epoch + 1)
        selected_losses.append(epoch_loss)
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    scheduler.step(epoch_loss)

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            if (epoch + 1) not in selected_epochs:
              selected_epochs.append(epoch + 1)
              selected_losses.append(epoch_loss)
            print(f"Early stopping at epoch {epoch + 1}, Loss: {epoch_loss:.4f}")
            break

end_time = time.time()
print(f"Χρόνος εκπαίδευσης: {(end_time - start_time) / 60:.2f} λεπτά")


plt.figure(figsize=(10, 6))
plt.plot(selected_epochs, selected_losses, label="Selected Loss", marker='o')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss σε Επιλεγμένες Εποχές")
plt.legend()
plt.grid()
plt.show()

model.eval()

def predict_acc(data, labels, name):
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == labels).sum().item()
        accuracy = 100 * correct / labels.size(0)

    print(f'Ακρίβεια στο {name} set: {accuracy:.2f}%')

    return predicted

predict_acc(train_data, train_labels, "Training")
predicted = predict_acc(test_data, test_labels, "Test")

def accuracy_per_category(test_labels, predicted, label_names):
    class_correct = []
    class_total = []

    for i in range(5):
        class_correct.append(0)
        class_total.append(0)

    for i in range(len(test_labels)):
        label = test_labels[i]
        class_total[label] += 1
        if predicted[i].item() == label:
            class_correct[label] += 1

    for i in range(5):
        if class_total[i] > 0:
          accuracy = 100 * class_correct[i] / class_total[i]
        else:
            accuracy = 0
        print(f"Κατηγορία: {label_names[i]:<10s} | Σωστά: {class_correct[i]:<3} / {class_total[i]:<3} | Ακρίβεια: {accuracy:.2f}%")

label_names =['snow', 'rain', 'fog', 'drizzle', 'sun']
accuracy_per_category(test_labels, predicted, label_names)

AttributeError: partially initialized module 'torch._dynamo' has no attribute 'config' (most likely due to a circular import)

In [ ]:
pip install seaborn

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

program_start_time = time.time()

dataset = pd.read_csv('seattle-weather.csv')
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

category_mapping = {category: idx for idx, category in enumerate(set(y), start=0)}

y = [category_mapping[weather] for weather in y]

print("Αντιστοίχιση κατηγοριών:", category_mapping)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

sc = StandardScaler()
train_data = sc.fit_transform(X_train)
test_data = sc.transform(X_test)

label_names = ['drizzle', 'rain', 'fog', 'snow', 'sun']

def accuracy_per_category(test_labels, predicted, label_names):
    class_correct = [0] * len(label_names)
    class_total = [0] * len(label_names)

    for i in range(len(test_labels)):
        label = test_labels[i]
        class_total[label] += 1
        if predicted[i] == label:
            class_correct[label] += 1

    for i in range(len(label_names)):
        accuracy = 100 * class_correct[i] / class_total[i] if class_total[i] > 0 else 0
        print(f"Κατηγορία: {label_names[i]:<10s} | Σωστά: {class_correct[i]:<3} / {class_total[i]:<3} | Ακρίβεια: {accuracy:.2f}%")

parameters = {
    'C': [100.0, 300.0, 500.0, 1000.0],
    'gamma': [0.002, 0.005, 0.01, 0.02, 0.05, 0.1],
}

model = SVC()
clf = GridSearchCV(estimator=model, param_grid=parameters, cv=10, scoring='accuracy')
clf.fit(train_data, y_train)

print("Best parameters set found on development set:")
print(clf.best_params_)

model = SVC(kernel='rbf', C=clf.best_params_['C'], gamma=clf.best_params_['gamma'])
model.fit(train_data, y_train)

predictions_train = model.predict(train_data)
predictions_test = model.predict(test_data)

print(f"Accuracy στο Training Set: {accuracy_score(y_true=y_train, y_pred=predictions_train)*100:.2f}%")
print(f"Accuracy στο Test Set: {accuracy_score(y_true=y_test, y_pred=predictions_test)*100:.2f}%")

accuracy_per_category(y_test, predictions_test, label_names)

program_end_time = time.time()
print(f"Χρόνος εκτέλεσης προγράμματος: {(program_end_time - program_start_time) / 60:.2f} λεπτά")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

program_start_time = time.time()

dataset = pd.read_csv('seattle-weather.csv')
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

category_mapping = {category: idx for idx, category in enumerate(set(y), start=0)}

y = [category_mapping[weather] for weather in y]

print("Αντιστοίχιση κατηγοριών:", category_mapping)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

sc = StandardScaler()
train_data = sc.fit_transform(X_train)
test_data = sc.transform(X_test)

label_names = ['drizzle', 'rain', 'fog', 'snow', 'sun']

classifier = LogisticRegression(random_state = 0, max_iter=1000)
classifier.fit(X_train, y_train)

yy = classifier.predict(X_train)
cm = confusion_matrix(y_train, yy)
print(cm)
print(f"Training set Accuracy: {accuracy_score(y_train, yy)*100:.2f}%")



y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(f"Test set Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%")